<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv',index_col=0)

In [3]:
corpus = data['text']

In [4]:
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
data.duplicated().sum()

0

m = Mystem()
def clear_text(text):
    text = text.lower()
    text = "".join(m.lemmatize(text))  
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.split()
    return " ".join(text)

In [7]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def clear_text(text):
    lemmatizer = WordNetLemmatizer()
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = word_tokenize(text)
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text]
    return " ".join(text)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)
print(df_my['text'].apply(clear_text))

                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked
0    the strip bat be hang on their foot for best
1       you should be ashamed of yourself go work
Name: text, dtype: object


In [10]:
%%time
data['text'] = data['text'].apply(clear_text)

CPU times: user 18min 32s, sys: 1min 38s, total: 20min 11s
Wall time: 20min 11s


In [11]:
data['text'].sample(10)

84849     vandalism why do you revert someone work on my...
5654      i wan na beeeeeeeeeeeeeer why wan na you give ...
149656    see no objection i m balance advertised number...
113265    easily divergent banias be about the spring an...
154241    overgrown outline the original article be inte...
88662     jimbo glad to see you comment here regardless ...
137822                with my finger up my nose just messin
74128     recent edits please remain civil and do not ma...
52382     welcome hello and welcome to wikipedia thank y...
119259    britton july be there some rule i m unaware of...
Name: text, dtype: object

## Обучение

Постройте модель со значением метрики качества F1 не меньше 0.75. 

In [12]:
features = data.drop('toxic', axis=1)
target = data['toxic']

In [13]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, stratify=target, random_state=12345)


In [14]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(127433, 1)
(127433,)
(31859, 1)
(31859,)


In [15]:
%%time
features_train = features_train['text']
params_lr = {'clf__C':[0.1, 1, 10]}

pipe_lr = Pipeline([('vect', CountVectorizer(stop_words=stopwords, binary=True)),
                 ('tfidf', TfidfTransformer()),
                 ('clf', LogisticRegression(class_weight='balanced', solver='lbfgs', max_iter=1000, random_state=12345))])

grid_lr = GridSearchCV(pipe_lr, params_lr, n_jobs = -1, cv=5, scoring = 'f1')
grid_lr.fit(features_train, target_train)
print("F1 на CV равен:", grid_lr.best_score_)

F1 на CV равен: 0.7627054023188515
CPU times: user 7min 59s, sys: 8min 25s, total: 16min 24s
Wall time: 16min 25s


In [16]:
%%time
param_rf = {'rf__max_depth': range(1,5),'rf__n_estimators': range(1,100,10)}

pipe_rf = Pipeline([('vect', CountVectorizer(stop_words=stopwords)),
                  ('tfidf', TfidfTransformer()),
                  ('rf', RandomForestClassifier(class_weight='balanced'))])

grid_rf = RandomizedSearchCV(pipe_rf, param_rf, n_jobs = -1, cv=5,
                               n_iter=1, scoring = 'f1')
grid_rf.fit(features_train, target_train)
print("F1 на CV равен:", grid_rf.best_score_)

F1 на CV равен: 0.20547082489498458
CPU times: user 36.7 s, sys: 475 ms, total: 37.2 s
Wall time: 37.3 s


Лучшей себя показала модель логической регресии. Проверим модель на тетсовых данных.

In [17]:
features_test_transformed = features_test['text']
target_pred = grid_lr.predict(features_test_transformed)

f1_test = f1_score(target_test, target_pred)
print("F1 на тестовых данных равен:", f1_test)

F1 на тестовых данных равен: 0.7648983074953776


Модель проходит критерий качества в F1 > 0.75 

## Выводы

В ходе выполнения проекта были выполнены основные операции:

Загруженны и расмотренны данные. Проведена предобработка текста комментариев, а так же датасет разделен на обучающую и тестовую выборку.

Проведено обучение моделей:
- LogisticRegression.
- RandomForestClassifier.

Рассчитано значение метрики F1 
Найдена лучшая модель и проверена на тестовой выборке.
Лучше всего себя показала модель LogisticRegression

Значение метрики F1 модели на тестовой выборке показало отличный результат (0.7648983074953776). Модель предсказывает со значением метрики качества F1 не меньше 0.75 и это значит что она подходит по критериям Заказчику.